# Otto Group Product Classification Challenge using nolearn/lasagne

This short notebook is meant to help you getting started with nolearn and lasagne in order to train a neural net and make a submission to the Otto Group Product Classification Challenge.

* [Otto Group Product Classification Challenge](https://www.kaggle.com/c/otto-group-product-classification-challenge)
* [Get the notebook from the Otto Group repository](https://github.com/ottogroup)
* [Nolearn repository](https://github.com/dnouri/nolearn)
* [Lasagne repository](https://github.com/benanne/Lasagne)
* [A nolearn/lasagne tutorial for convolutional nets](http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial/)

## Imports

In [59]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [60]:
from lasagne.layers import DenseLayer
from lasagne.layers import InputLayer
from lasagne.layers import DropoutLayer
from lasagne.nonlinearities import softmax
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet

## Utility functions

In [61]:
def load_train_data(path):
    df = pd.read_csv(path)
    X = df.values.copy()
    np.random.shuffle(X)
    X, labels = X[:, 1:-1].astype(np.float32), X[:, -1]
    encoder = LabelEncoder()
    y = encoder.fit_transform(labels).astype(np.int32)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return X, y, encoder, scaler

In [62]:
def load_test_data(path, scaler):
    df = pd.read_csv(path)
    X = df.values.copy()
    X, ids = X[:, 1:].astype(np.float32), X[:, 0].astype(str)
    X = scaler.transform(X)
    return X, ids

In [63]:
def make_submission(clf, X_test, ids, encoder, name='my_neural_net_submission.csv'):
    y_prob = clf.predict_proba(X_test)
    with open(name, 'w') as f:
        f.write('id,')
        f.write(','.join(encoder.classes_))
        f.write('\n')
        for id, probs in zip(ids, y_prob):
            probas = ','.join([id] + map(str, probs.tolist()))
            f.write(probas)
            f.write('\n')
    print("Wrote submission to file {}.".format(name))

## Load Data

In [71]:
X, y, encoder, scaler = load_train_data('data/train234.csv')

In [72]:
X_test, ids = load_test_data('difficult_cases.csv', scaler)

In [73]:
num_classes = len(encoder.classes_)
num_features = X.shape[1]

In [74]:
X

array([[-0.24935143, -0.10235909,  5.0120101 , ..., -0.11613525,
        -0.3592934 , -0.12855771],
       [-0.24935143, -0.10235909, -0.23580633, ..., -0.11613525,
         1.38053524, -0.12855771],
       [-0.24935143, -0.10235909, -0.23580633, ..., -0.11613525,
        -0.3592934 , -0.12855771],
       ..., 
       [-0.24935143, -0.10235909, -0.23580633, ..., -0.11613525,
        -0.3592934 , -0.12855771],
       [-0.24935143, -0.10235909, -0.23580633, ..., -0.11613525,
        -0.3592934 , -0.12855771],
       [-0.24935143, -0.10235909, -0.23580633, ..., -0.11613525,
        -0.3592934 , -0.12855771]], dtype=float32)

## Train Neural Net

In [86]:
layers0 = [('input', InputLayer),
           ('dense0', DenseLayer),
           ('dropout1', DropoutLayer),
           ('dense1', DenseLayer),
           #('dropout2', DropoutLayer),
           #('dense2', DenseLayer),
           ('output', DenseLayer)]

In [93]:
net0 = NeuralNet(layers=layers0,
                 
                 input_shape=(None, num_features),
                 
                 dense0_num_units=200,
                 
                 dropout1_p=0.7,
                 
                 dense1_num_units=200,
                 
                 #dropout2_p=0.5,
                 
                 #dense2_num_units=300,
                 
                 
                 output_num_units=num_classes,
                 output_nonlinearity=softmax,
                 
                 update=nesterov_momentum,
                 update_learning_rate=0.001,
                 update_momentum=0.9,
                 
                 eval_size=0.2,
                 verbose=1,
                 max_epochs=1000)


In [94]:
net0.fit(X, y)

  InputLayer        	(None, 93)          	produces      93 outputs
  DenseLayer        	(None, 200)         	produces     200 outputs
  DropoutLayer      	(None, 200)         	produces     200 outputs
  DenseLayer        	(None, 200)         	produces     200 outputs
  DenseLayer        	(None, 3)           	produces       3 outputs

 Epoch  |  Train loss  |  Valid loss  |  Train / Val  |  Valid acc  |  Dur
--------|--------------|--------------|---------------|-------------|-------
     1  |    1.050487  |    0.869532  |     1.208105  |     61.08%  |  2.1s
     2  |    0.919342  |    0.830289  |     1.107257  |     62.26%  |  2.1s
     3  |    0.880981  |    0.810604  |     1.086821  |     63.18%  |  2.3s
     4  |    0.857259  |    0.796626  |     1.076112  |     63.46%  |  2.2s
     5  |    0.847587  |    0.785711  |     1.078752  |     63.95%  |  2.1s
     6  |    0.836164  |    0.777518  |     1.075427  |     64.58%  |  2.1s
     7  |    0.823430  |    0.769542  |     1.070026  | 

NeuralNet(X_tensor_type=<function matrix at 0x7f377927f140>,
     batch_iterator_test=<nolearn.lasagne.BatchIterator object at 0x7f3775e3dd50>,
     batch_iterator_train=<nolearn.lasagne.BatchIterator object at 0x7f3775e3dd10>,
     dense0_num_units=200, dense1_num_units=200, dropout1_p=0.7,
     eval_size=0.2, input_shape=(None, 93),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('dense0', <class 'lasagne.layers.dense.DenseLayer'>), ('dropout1', <class 'lasagne.layers.noise.DropoutLayer'>), ('dense1', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=<function negative_log_likelihood at 0x7f3776761668>,
     max_epochs=1000, more_params={}, on_epoch_finished=(),
     on_training_finished=(),
     output_nonlinearity=<theano.tensor.nnet.nnet.Softmax object at 0x7f3778ec9b90>,
     output_num_units=3, regression=False,
     update=<function nesterov_momentum at 0x7f3776761320>,
     update_learning_rate=0.0

In [46]:
make_submission(net0, X_test, ids, encoder, "what.csv")

Wrote submission to file what.csv.
